### A3

Wir lesen ein 3D-Grid ein, das aus zwei Ebenen besteht.

In [1]:
%%writefile input.txt
3 5
#####
# # #
#####

#####
#   #
#####

Overwriting input.txt


In [2]:
f = open('input.txt')
hoehe, breite = [int(x) for x in f.readline().split()]

grid0 = [list(f.readline().strip()) for _ in range(hoehe)]
f.readline()    # leerzeile
grid1 = [list(f.readline().strip()) for _ in range(hoehe)]
f.close()
grid = [grid0,grid1]

In [3]:
def show(grid):
    for row in grid[0]:
        print(*row,sep='')
    print()
    for row in grid[1]:
        print(*row,sep='')

show(grid)

#####
# # #
#####

#####
#   #
#####


Wir können mit Indizes auf die Elemente des 3D-grids zugreifen. Der erste Index ist die Ebene, Index 0 ist die oberste Ebene.

In [4]:
grid[0][1][3] = 'A'   # oberste Ebene, Zeile 1, Spalte 3
show(grid)

#####
# #A#
#####

#####
#   #
#####


Wir können das grid in einen gewichteten Graphen umwandeln, bei dem wir berücksichtigen, dass der Ebenenwechsel mit höherenn Kosten verbunden ist. Die Knoten V des Graphen sind die Koordinaten der begehbaren Punkte.

In [5]:
V = [(z,x,y) for z in range(2) for x in range(hoehe) for y in range(breite) if grid[z][x][y] != '#']
V

[(0, 1, 1), (0, 1, 3), (1, 1, 1), (1, 1, 2), (1, 1, 3)]

Jedem Knoten werden seine Nachbarknoten mit den Kosten zugeordnet.

In [6]:
G = {v: dict() for v in V}
dirs = [(1,0,0),(-1,0,0),(0,1,0),(0,-1,0),(0,0,1),(0,0,-1)]   # mögliche Richtungen
for v in V:
    z,x,y = v
    for zd,xd,yd in dirs:
        zn, xn, yn = z+zd, x+xd, y+yd
        if 0 <= zn < 2 and 0 <= xn < hoehe and 0 <= zn < breite and grid[zn][xn][yn] != '#':
            posn = (zn,xn,yn)
            G[v][posn] = 1 if z == zn else 3

G[(1,1,1)]     

{(0, 1, 1): 3, (1, 1, 2): 1}

G[(1,1,1)] liefert
{(0, 1, 1): 3, (1, 1, 2): 1}, d.h. wir können von (1,1,1) mit Kosten 3 nach (0,1,1) und mit Kosten 1 nach (1,1,2)

Eine Implementation des Dijkstra-Algorithmus findet sich im Abschnitt Infokurs/Graphen1